In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf

# Set the path to the dataset
data_folder = "C:\\Users\\notif\\Downloads\\archive (3)"

# Set the input size of the images
input_size = (64, 64)

def load_dataset():
    images = []
    labels = []
    for mood in os.listdir(data_folder):
        mood_folder = os.path.join(data_folder, mood)
        for image_file in os.listdir(mood_folder):
            image_path = os.path.join(mood_folder, image_file)
            try:
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Failed to read image: {image_path}")
                    continue
                image = cv2.resize(image, input_size)
                images.append(image)
                labels.append(mood)
            except Exception as e:
                print(f"Error processing image: {image_path}")
                print(str(e))
    return np.array(images), np.array(labels)

# Preprocess the data
def preprocess_data(images, labels):
    # Normalize pixel values
    images = images.astype('float32') / 255.0

    # Perform one-hot encoding on labels
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)

    # Split the dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

    return X_train, X_test, y_train, y_test

# Build the model
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model
def train_model(model, X_train, y_train, epochs=10, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

# Load and preprocess the dataset
images, labels = load_dataset()
X_train, X_test, y_train, y_test = preprocess_data(images, labels)

# Build the model
input_shape = (input_size[0], input_size[1], 3)  # Assuming RGB images
num_classes = len(np.unique(labels))
model = build_model(input_shape, num_classes)

# Train the model
train_model(model, X_train, y_train)

# Evaluate the model
evaluate_model(model, X_test, y_test)

# Use the model for song recommendation on new images
def suggest_songs(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, input_size)
    image = np.expand_dims(image, axis=0)
    image = image.astype('float32') / 255.0

    # Predict the mood category
    predicted_label = model.predict(image)
    predicted_mood = label_encoder.inverse_transform(np.argmax(predicted_label))

    # Retrieve the playlist for the predicted mood and suggest songs
    playlist = get_playlist_for_mood(predicted_mood)
    suggest_songs_to_user(playlist)

# Example usage for song suggestion on a new image
new_image_path = "path/to/new/image.jpg"
suggest_songs(new_image_path)

Failed to read image: C:\Users\notif\Downloads\archive (3)\test\angry
Failed to read image: C:\Users\notif\Downloads\archive (3)\test\disgust
Failed to read image: C:\Users\notif\Downloads\archive (3)\test\fear
Failed to read image: C:\Users\notif\Downloads\archive (3)\test\happy
Failed to read image: C:\Users\notif\Downloads\archive (3)\test\neutral
Failed to read image: C:\Users\notif\Downloads\archive (3)\test\sad
Failed to read image: C:\Users\notif\Downloads\archive (3)\test\surprise
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\angry
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\disgust
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\fear
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\happy
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\neutral
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\sad
Failed to read image: C:\Users\notif\Downloads\archive (3)\train\surprise


ValueError: zero-size array to reduction operation maximum which has no identity